# Run Union Workflows on a Jupyter Notebook

First create a Union Serverless account:
👉 https://signup.union.ai/

First install the dependencies:

In [ ]:
%pip install "flytekit==1.14.0b5" union "pydantic>2"

Then, login to Union on this notebook session:

In [ ]:
!union create login --auth device-flow --serverless

## Create some data

Let's create some toy data:

In [56]:
import flytekit as fk
import sys


image = fk.ImageSpec(
    name="jupyter-notebook-workshop",
    packages=["flytekit==1.14.0b5"],
    python_version=f"{sys.version_info.major}.{sys.version_info.minor}",
)


task = fk.task(container_image=image)


@task
def create_dataset(n: int) -> list[int]:
    return [*range(n)]


@fk.workflow
def dataset_wf(n: int = 100) -> list[int]:
    return create_dataset(n)

## Running on Union

Let's define a `UnionRemote` object:

In [ ]:
from union.remote import UnionRemote

serverless = UnionRemote()

Then, we can execute the workflow like so:

In [ ]:
dataset_exec = serverless.execute(dataset_wf, inputs={"n": 50})
print(dataset_exec.execution_url)

We can fetch the output of the workflow back into our notebook:

In [ ]:
# Wait for the task to complete
dataset_exec = dataset_exec.wait(poll_interval=1)

# Print the outputs
print(dataset_exec.outputs)

## Compute the mean

We can then pass the data back into another workflow that computes the mean:

In [60]:
@task
def compute_mean(data: list[int]) -> float:
    return sum(data) / len(data)


@fk.workflow
def mean_wf(data: list[int]) -> float:
    return compute_mean(data)

Run the workflow:

In [ ]:
# Execute the workflow
data, *_ = dataset_exec.outputs.values()
mean_wf_exec = serverless.execute(mean_wf, inputs={"data": data})
print(mean_wf_exec.execution_url)

In [ ]:
# Wait for the task to complete
mean_wf_exec = mean_wf_exec.wait(poll_interval=1)

# Print the outputs
print(mean_wf_exec.outputs)

## Parallelizing with `map_task`

We can also parallelize the computation of the mean by using the `map_task` method:

In [63]:
map_compute_mean = fk.map_task(compute_mean)

@fk.workflow
def parallel_mean_wf(data_list: list[list[int]]) -> list[float]:
    return map_compute_mean(data=data_list)

Execute the workflow:

In [ ]:
parallel_mean_wf_exec = serverless.execute(parallel_mean_wf, inputs={"data_list": [[1, 2, 3], [2, 3, 4], [3, 4, 5]]})
print(parallel_mean_wf_exec.execution_url)

In [ ]:
# Wait for the task to complete
parallel_mean_wf_exec = parallel_mean_wf_exec.wait(poll_interval=1)

# Print the outputs
print(parallel_mean_wf_exec.outputs)

And that's it! You can now iterate on your workflows and tasks directly inside a Jupyter Notebook 🚀